In [2]:
!pip install sacrebleu
!pip install datasets
!pip install tqdm
!pip install numpy
!pip install huggingface_hub
!pip install faster-whisper -q
!pip install ctranslate2==4.6.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

In [3]:
!ct2-transformers-converter --model farahabdou/whisper-arabic-english --output_dir farahabdou/whisper-arabic-english-eval --copy_files tokenizer.json preprocessor_config.json --quantization float16

config.json: 100% 1.38k/1.38k [00:00<00:00, 5.88MB/s]
2024-12-23 15:56:19.588120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-23 15:56:19.623261: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-23 15:56:19.633769: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-23 15:56:19.657884: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compile

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from faster_whisper import WhisperModel
from sacrebleu.metrics import BLEU, CHRF, TER
from datasets import load_dataset, Dataset, Audio, DatasetDict
from tqdm import tqdm
import numpy as np


In [6]:
fleurs_dataset = load_dataset("farahabdou/FLEURS-AR-EN-split",trust_remote_code=True)

README.md:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/489M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/492M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/492M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/186M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2228 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/278 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/279 [00:00<?, ? examples/s]

In [ ]:
def predict(audio_array):
    model_name = "base"  # pretrained model - try "tiny", "base", "small", or "medium"
    model = WhisperModel(model_name, device="cpu")

    segments, info = model.transcribe(audio_array,
                                      beam_size=5,
                                      language="en",
                                      vad_filter=True
                                      )
    return segments, info

def metric_calculation(dataset):
    val_data = fleurs_dataset["validation"]
    bleu = BLEU()
    chrf = CHRF()
    lst_pred = []
    lst_gold = []
    for data in tqdm(val_data):
        gold_standard = data["english"]
        gold_standard = gold_standard.lower().strip()
        audio_array = data["audio"]["array"]
        # Ensure it's 1D
        audio_array = np.ravel(audio_array)

        # Convert to float32 if necessary
        audio_array = audio_array.astype(np.float32)
        pred_segments, pred_info = predict(audio_array)
        prediction_text = " ".join(segment.text for segment in pred_segments).lower().strip()
        lst_pred.append(prediction_text)
        lst_gold.append([gold_standard])
    bleu_score = bleu.corpus_score(lst_pred, lst_gold).score
    chrf_score = chrf.corpus_score(lst_pred, lst_gold).score

    return bleu_score, chrf_score

In [ ]:
final_bleu_score, final_chrf_score = metric_calculation(fleurs_dataset)
final_bleu_score, final_chrf_score

  0%|          | 0/278 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
final_bleu_score, final_chrf_score

In [7]:
import os
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
import subprocess
import gc

def convert_whisper_model(model_id, output_dir):
    try:
        # Free memory
        torch.cuda.empty_cache()
        gc.collect()

        # Load model with half precision
        model = AutoModelForSpeechSeq2Seq.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            use_safetensors=True
        )

        processor = AutoProcessor.from_pretrained(model_id)

        # Create output directory
        os.makedirs(output_dir, exist_ok=True)

        # Save model and processor
        model.save_pretrained(output_dir)
        processor.save_pretrained(output_dir)

        # Free memory
        del model, processor
        torch.cuda.empty_cache()
        gc.collect()

        # Convert using ctranslate2
        subprocess.run([
            'ct2-transformers-converter',
            '--model', output_dir,
            '--output_dir', output_dir,
            '--quantization', 'float16',
            '--force'
        ], check=True)

        print("Conversion completed successfully!")

    except Exception as e:
        print(f"Error during conversion: {str(e)}")

# Usage
model_id = "farahabdou/whisper-arabic-english"
output_dir = "whisper-arabic-english-ct2"
convert_whisper_model(model_id, output_dir)

Conversion completed successfully!


In [8]:
from faster_whisper import WhisperModel
ctranslate2_dir = "farahabdou/whisper-arabic-english-eval"
def predict(audio_array):
    model = WhisperModel(ctranslate2_dir, device="cpu")  # Load the converted model

    segments, info = model.transcribe(audio_array,
                                      beam_size=5,
                                      language="en",
                                      vad_filter=True
                                      )
    return segments, info

def metric_calculation(dataset):
    val_data = fleurs_dataset["validation"]
    bleu = BLEU()
    chrf = CHRF()
    lst_pred = []
    lst_gold = []
    for data in tqdm(val_data):
        gold_standard = data["english"]
        gold_standard = gold_standard.lower().strip()
        audio_array = data["audio"]["array"]
        # Ensure it's 1D
        audio_array = np.ravel(audio_array)

        # Convert to float32 if necessary
        audio_array = audio_array.astype(np.float32)
        pred_segments, pred_info = predict(audio_array)
        prediction_text = " ".join(segment.text for segment in pred_segments).lower().strip()
        lst_pred.append(prediction_text)
        lst_gold.append([gold_standard])
    bleu_score = bleu.corpus_score(lst_pred, lst_gold).score
    chrf_score = chrf.corpus_score(lst_pred, lst_gold).score

    return bleu_score, chrf_score

In [ ]:
pretrain_bleu_score, pretrain_chrf_score   = metric_calculation(fleurs_dataset)

pretrain_bleu_score, pretrain_chrf_score

 37%|███▋      | 102/278 [55:33<1:05:26, 22.31s/it]

In [ ]:
pretrain_bleu_score, pretrain_chrf_score

In [ ]:
!sudo rm -rf /root/.cache/huggingface/hub/
!sudo mkdir /root/.cache/huggingface/

In [ ]:
!sudo rm -rf  /content/whisper-small-idn-ct

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model = WhisperForConditionalGeneration.from_pretrained("sanchit-gandhi/whisper-small-hi")
processor = WhisperProcessor.from_pretrained("sanchit-gandhi/whisper-small-hi")